The main solving big ridge regression problems

In [ ]:
workspace()

In [ ]:
using Plots

In [313]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [314]:
#Basic parameters
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [317]:
dataset = "gisette_scale"   # news20.binary  rcv1_train.binary mnist.scale  a9a  gisette_scale  rcv1_train.binary
prob = defineProb(dataset);

../data/gisette_scale.jld
ErrorException("File ../data/gisette_scale-xsol.jld cannot be found")


In [318]:
norm(prob.A*prob.xsol - prob.b)
showDetails(dataset)

LoadError: KeyError: key "gisette_scale" not found

In [319]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [ ]:
options.AUX = [0 1]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 500; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 100; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 10; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 500]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 1; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [326]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =10; # Int(floor(log2(prob.n)));
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 161000
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
 13  | 52.34  |  17.49  | 202.0304 
202.149252 seconds (2.19 G allocations: 354.684 GB, 46.83% gc time)


In [325]:
# Testing countmin 
options.sketchsize =10;#floor((prob.n)^(1/2));
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

countmin
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 48.37  |  13.89  | 6.3251 
 50  | 41.60  |  9.77  | 11.5942 
 75  | 37.19  |  8.57  | 16.8104 
100  | 33.78  |  8.55  | 22.1589 
125  | 31.68  |  7.18  | 27.3806 
150  | 29.63  |  7.18  | 32.7585 
175  | 27.53  |  6.81  | 38.0001 
200  | 25.57  |  5.97  | 43.2613 
225  | 24.93  |  5.46  | 48.4440 
250  | 22.70  |  5.81  | 53.7164 
275  | 21.17  |  5.96  | 59.6698 
300  | 19.88  |  4.50  | 65.9875 
325  | 18.37  |  4.76  | 71.1524 
350  | 17.45  |  4.15  | 76.4680 
375  | 17.65  |  3.91  | 81.5909 
400  | 16.72  |  3.86  | 86.9078 
425  | 15.86  |  3.78  | 92.1332 
450  | 15.49  |  3.31  | 97.4837 
475  | 14.76  |  3.58  | 102.7025 
500  | 14.60  |  3.12  | 108.0037 
525  | 13.80  |  3.44  | 113.2106 
550  | 13.40  |  2.97  | 118.5170 
575  | 12.35  |  3.04  | 123.8673 
600  | 12.27  |  3.05  | 129.0206 
625  | 12.17  |  3.69  | 134.4015 
650  | 11.45  |  3.67  | 139.6169 
675  | 11.53  |  2.63  

In [321]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 3.89  |  1.53  | 0.6564 
 50  | 1.76  |  0.67  | 1.2130 
 75  | 1.10  |  0.49  | 1.7700 
100  | 0.77  |  1.21  | 2.3274 
125  | 0.57  |  0.25  | 2.8970 
150  | 0.30  |  0.38  | 3.4821 
175  | 0.92  |  0.18  | 4.0527 
200  | 0.48  |  0.20  | 4.5608 
225  | 0.23  |  0.13  | 5.0683 
250  | 0.19  |  0.82  | 5.5513 
275  | 0.16  |  0.22  | 6.0433 
300  | 0.13  |  0.16  | 6.5322 
325  | 0.11  |  0.22  | 7.0176 
350  | 0.09  |  0.16  | 7.4789 
375  | 0.02  |  0.10  | 7.9893 
400  | 0.07  |  0.11  | 8.4833 
425  | 0.05  |  0.29  | 8.9930 
450  | 0.04  |  0.07  | 9.5145 
475  | 0.04  |  0.07  | 10.0201 
500  | 0.03  |  0.04  | 10.5416 
525  | 0.03  |  0.04  | 11.0502 
550  | 0.02  |  0.03  | 11.5566 
575  | 0.02  |  0.08  | 12.0656 
600  | 0.02  |  0.03  | 12.5730 
625  | 0.09  |  0.03  | 13.2212 
650  | 0.05  |  0.03  | 13.7721 
675  | 0.06  |  0.03  | 14.2876 
700  | 0.06  |  0.03  | 14.7978 
725  | 0.01  | 

In [324]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = 200;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-200
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 9.17  |  2.93  | 0.5781 
 50  | 5.60  |  1.26  | 0.9956 
 75  | 4.04  |  0.91  | 1.3728 
100  | 3.14  |  0.69  | 1.7714 
125  | 2.98  |  0.58  | 2.2041 
150  | 2.01  |  0.50  | 2.5667 
175  | 1.97  |  0.44  | 2.9538 
200  | 1.73  |  0.39  | 3.3187 
225  | 1.56  |  0.35  | 3.7083 
250  | 1.42  |  0.31  | 4.0870 
275  | 1.43  |  0.29  | 4.5077 
300  | 1.30  |  0.25  | 4.8632 
325  | 1.11  |  0.25  | 5.2580 
350  | 1.24  |  0.22  | 5.6639 
375  | 0.95  |  0.22  | 6.0942 
400  | 0.80  |  0.19  | 6.7400 
425  | 0.48  |  0.20  | 7.3766 
450  | 0.81  |  0.19  | 7.9461 
475  | 0.77  |  0.18  | 8.3899 
500  | 0.73  |  0.17  | 8.7675 
525  | 0.69  |  0.16  | 9.3750 
550  | 0.83  |  0.14  | 9.7314 
575  | 0.48  |  0.14  | 10.0909 
600  | 0.80  |  0.14  | 10.4560 
625  | 0.29  |  0.13  | 10.8139 
650  | 0.68  |  0.13  | 11.1680 
675  | 0.54  |  0.12  | 11.7558 
700  | 0.51  |  0.13  | 12.1280 
725  | 0.46  | 

In [293]:
OUTPUTS =[];
include("../src/call_all_methods.jl")

rademacher-10-10-sgn
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 22.94  |  10.80  | 0.0881 
 50  | 15.23  |  7.68  | 0.1183 
 75  | 11.62  |  7.23  | 0.1505 
100  | 9.47  |  4.28  | 0.1775 
125  | 7.86  |  3.68  | 0.2122 
150  | 6.90  |  3.49  | 0.2398 
175  | 6.52  |  2.95  | 0.2746 
200  | 5.38  |  2.47  | 0.3025 
225  | 4.88  |  2.34  | 0.3327 
250  | 4.47  |  2.43  | 0.3597 
275  | 4.73  |  2.09  | 0.3896 
300  | 3.74  |  1.73  | 0.4171 
325  | 3.48  |  1.89  | 0.4473 
350  | 3.26  |  1.47  | 0.4743 
375  | 3.93  |  1.84  | 0.5042 
400  | 2.87  |  1.85  | 0.5317 
425  | 2.72  |  1.36  | 0.5617 
450  | 2.57  |  1.23  | 0.5888 
475  | 2.24  |  1.35  | 0.6186 
500  | 2.32  |  1.52  | 0.6449 
525  | 2.18  |  1.07  | 0.6730 
550  | 2.08  |  0.96  | 0.6965 
575  | 1.99  |  1.03  | 0.7226 
600  | 1.90  |  1.01  | 0.7472 
625  | 1.82  |  1.02  | 0.7737 
650  | 1.75  |  0.86  | 0.7974 
675  | 1.28  |  0.92  | 0.8234 
700  | 1.64  |  0.80  | 0.8490 
725

4-element Array{Any,1}:
 LastMain.Output(1000,53085,[0.0 0.0880732 … 1.13277 1.16704],[1.0 0.229355 … 0.011472 0.0112244],[1.0 0.108 … 0.0054965 0.00548314],"rademacher-10-10-sgn","max_iter") 
 LastMain.Output(1000,53085,[0.0 0.0158351 … 0.582131 0.59705],[1.0 0.234526 … 0.0131045 0.0127203],[1.0 0.11109 … 0.00662151 0.00636297],"rademacher-10-10","max_iter")
 LastMain.Output(1000,1222260,[0.0 0.00193524 … 0.274848 0.277471],[1.0 0.0124964 … 0.000123463 0.000118088],[1.0 0.0294147 … 0.000262751 0.000209458],"CG","max_iter") 
 LastMain.Output(1000,8820,[0.0 0.00185541 … 0.054431 0.0555454],[1.0 0.266163 … 0.00702874 0.00662517],[1.0 0.166018 … 0.00757432 0.00744022],"CD-10","max_iter")      

In [305]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [306]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.binary


In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];